In [1]:
import numpy as np
import pandas as pd
import math
from itertools import count
from tqdm import tqdm
from functools import cmp_to_key
import random as random

In [2]:
# i-th row of a n*n identity matrix
def eyerow(n, i):
    row = np.zeros(n, dtype=int)
    row[i] = 1
    return row

# degree of a vector
def degree(e):
    return np.sum(e)

def vec2str(v):
    return '[' + (''.join([' %2d'%i for i in v])) + ']'

def dual2vec(e,l):
    return vec2str(e) + vec2str(l) + ' deg:' + str(degree(e))

def printDual(EL, verbose=True, end='\n'):
    if verbose:
        for e,l in EL:
            print(vec2str(e)+vec2str(l), end=end)
        print()

def sort_rows(mat):
    return np.array(sorted(mat, key=lambda row: tuple(row)))

# read matrix from file in 4ti2 format
def read_mat(fname):
    f = open(fname)
    nr,nc = f.readline().split(' ')
    nr,nc = int(nr), int(nc)
    if nr==0:
        return np.zeros([nr,nc], dtype=int)
    return np.loadtxt(f, dtype=int)

# ComputePHC using Hilbert bases

In [3]:
# graded order comparator
def deglex_compare(e1l1, e2l2):
    e1,l1 = e1l1 ; e2,l2 = e2l2
    # degree comparison
    deg1, deg2 = degree(e1), degree(e2)
    if   deg1 < deg2: return -1
    elif deg1 > deg2: return +1
    # finds index of the first non matching element
    idx = np.where(e1 != e2)[0]
    if len(idx)==0: return 0 # same vectors
    elif e1[idx[0]] < e2[idx[0]]: return -1
    elif e1[idx[0]] > e2[idx[0]]: return +1
    assert False

# partial order comparison
def po_compare(e1l1, e2l2, j):
    e1,l1 = e1l1 ; e2,l2 = e2l2
    ll1, ll2 = l1[j], l2[j]
    return ll1*ll2 >= 0 and abs(ll1) <= abs(ll2) and np.less_equal(e1, e2).all()

# append a vector to a set (testing for uniqueness)
def append_unique(C, e,l):
    insert = True
    for ex, lx in C:
        if np.equal(e, ex).all() and np.equal(l, lx).all():
            return False # already exists
    C.append( (e,l) )
    return True

# combine all C+ and C- vectors
def S_vectors(Cp, Cm, j):
    G = []
    for ep, lp in Cp:
        for em, lm in Cm:
            e, l = ep + em, lp + lm
            assert abs(l[j]) < abs(lp[j]) + abs(lm[j])
            append_unique(G, e,l)
    return G

# # check a-minimality
def is_irreducible(e,l, C0,Cp,Cm, j):
    for e0,l0 in C0:
        if po_compare( (e0,l0), (e,l), j): 
            return False
    for e0,l0 in Cp:
        if po_compare( (e0,l0), (e,l), j): 
            return False
    for e0,l0 in Cm:
        if po_compare( (e0,l0), (e,l), j): 
            return False
    return True

# Compose PHC on column j
def ComputePHC(EL, j, verbose):
    if verbose: print(f'\n------------- j={j}, |EL|={len(EL)} --------------'); printDual(EL)
    Cp, Cm, C0 = [], [], []
    G = [(e,l) for e,l in EL]
    while len(G) > 0:
        G = sorted(G, key=cmp_to_key(deglex_compare))
        e,l = G[0]
        del G[0]
        if is_irreducible(e,l, C0,Cp,Cm, j):
            if l[j]==0:
                C0.append((e,l))
                if verbose: print('C0 <- C0 U',dual2vec(e,l))
            elif l[j]>0: 
                Cp.append((e,l))
                G.extend(S_vectors([(e,l)], Cm, j))
                if verbose: print('C+ <- C+ U',dual2vec(e,l))
            elif l[j]<0: 
                Cm.append((e,l))
                G.extend(S_vectors(Cp, [(e,l)], j))
                if verbose: print('C- <- C- U',dual2vec(e,l))
        else:
            if verbose: print('DROP      ',dual2vec(e,l))
    # Build the result
    D = C0 + Cp + Cm
    for e,l in EL:
        append_unique(D, e,l)
    return D

In [4]:
mat = [ [1], [2], [-2] ]
print(np.array(mat),'\n')

# Build dual matrix [E|L]
EL = [ (eyerow(len(mat),i), np.array(mat[i])) for i in range(len(mat))]
# Compose PHC on the selected column
H = ComputePHC(EL, 0, verbose=True)

print('\nResult:')
printDual(H)

[[ 1]
 [ 2]
 [-2]] 


------------- j=0, |EL|=3 --------------
[  1  0  0][  1]
[  0  1  0][  2]
[  0  0  1][ -2]

C- <- C- U [  0  0  1][ -2] deg:1
C+ <- C+ U [  0  1  0][  2] deg:1
C+ <- C+ U [  1  0  0][  1] deg:1
C0 <- C0 U [  0  1  1][  0] deg:2
C- <- C- U [  1  0  1][ -1] deg:2
DROP       [  1  1  1][  1] deg:3
C0 <- C0 U [  2  0  1][  0] deg:3

Result:
[  0  1  1][  0]
[  2  0  1][  0]
[  0  1  0][  2]
[  1  0  0][  1]
[  0  0  1][ -2]
[  1  0  1][ -1]

